1. Create a Zillow Api account to obtain a zillow key
2. Register with RapidApi to access unnofficial Zillow Api

In [1]:
#Import all necessary libraries 

import pandas as pd 
import requests 
import json
import time
import csv

In [2]:
#Importing all data and API's
df = pd.read_csv('realestateproject.csv')
zwid = df.loc[df['API']== 'zillow']['Key'].iloc[0]
rapid = df.loc[df['API']== 'Rapid']['Key'].iloc[0]

#Show all columns in display
pd.options.display.max_columns = None 

In [3]:
# Specify what data you want to look at 

city = 'charlotte'
state = 'nc'
search_str = city + ', ' + state
print('Search string:', search_str)

Search string: charlotte, nc


In [4]:
# Add search API

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_str,"home_type":"Houses",
               "minPrice": "150000",
               "maxPrice": "400000",
               "sqftMin": "1300"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": rapid
}

for_sale = requests.request("GET", url, headers=headers, params=querystring)

# transform to Json
for_sale_json = for_sale.json()
for_sale_json

{'props': [{'dateSold': None,
   'propertyType': 'SINGLE_FAMILY',
   'lotAreaValue': 10541.52,
   'address': '4621 Hoste Way Ln, Charlotte, NC 28216',
   'daysOnZillow': -1,
   'price': 360000,
   'listingDateTime': None,
   'longitude': -80.93323,
   'latitude': 35.33016,
   'contingentListingType': None,
   'listingStatus': 'FOR_SALE',
   'zpid': '64999714',
   'listingSubType': {'is_FSBA': True},
   'imgSrc': 'https://photos.zillowstatic.com/fp/55bc510c0de287475ed4886aecdb7e42-p_e.jpg',
   'livingArea': 2521,
   'bathrooms': 3,
   'lotAreaUnit': 'sqft',
   'country': 'USA',
   'currency': 'USD',
   'bedrooms': 4,
   'hasImage': True},
  {'dateSold': None,
   'propertyType': 'SINGLE_FAMILY',
   'lotAreaValue': 9234.72,
   'address': '425 Heflin St, Charlotte, NC 28205',
   'daysOnZillow': -1,
   'price': 300000,
   'listingDateTime': None,
   'longitude': -80.80592,
   'latitude': 35.202858,
   'contingentListingType': None,
   'listingStatus': 'FOR_SALE',
   'zpid': '299908304',
   

In [5]:
# Turn data into a dataframe
df = pd.json_normalize(data=for_sale_json['props'])
df.head()

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_comingSoon,listingSubType.is_openHouse,listingSubType.is_newHome
0,None,SINGLE_FAMILY,10541.52,"4621 Hoste Way Ln, Charlotte, NC 28216",-1,360000,None,-80.93323,35.330160,None,FOR_SALE,64999714,https://photos.zillowstatic.com/fp/55bc510c0de...,2521,3,sqft,USA,USD,4,True,True,NaN,NaN,NaN
1,None,SINGLE_FAMILY,9234.72,"425 Heflin St, Charlotte, NC 28205",-1,300000,None,-80.80592,35.202858,None,FOR_SALE,299908304,https://photos.zillowstatic.com/fp/216963841e6...,1454,2,sqft,USA,USD,3,True,NaN,True,NaN,NaN
2,None,SINGLE_FAMILY,1.06,"7230 Ridge Lane Rd, Charlotte, NC 28262",-1,350000,None,-80.77863,35.304090,None,FOR_SALE,6161650,https://photos.zillowstatic.com/fp/b8e8eb53f72...,1729,2,acres,USA,USD,3,True,True,NaN,True,NaN
3,None,SINGLE_FAMILY,6011.28,"2926 Royal Fern Ln, Charlotte, NC 28215",-1,369900,None,-80.72796,35.253464,None,FOR_SALE,80448507,https://photos.zillowstatic.com/fp/dee8e4d5ff4...,4140,4,sqft,USA,USD,5,True,True,NaN,NaN,NaN
4,None,SINGLE_FAMILY,4356.00,"5918 Bayswater Ln, Charlotte, NC 28212",-1,379900,None,-80.74150,35.219400,None,FOR_SALE,6203929,https://photos.zillowstatic.com/fp/e9693481b6f...,1511,3,sqft,USA,USD,4,True,True,NaN,NaN,NaN


In [24]:
# Create empty list 
property_detail_list = [ ]

# Get property details by importing property details API

zpid_list = df['zpid'].tolist()

for zpid in zpid_list:
    url = "https://zillow-com1.p.rapidapi.com/property"

    querystring = {"zpid":zpid}

    headers = {
        "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
        "X-RapidAPI-Key": rapid
    }

    property_details = requests.request("GET", url, headers=headers, params=querystring)
    pd_json = property_details.json()
    
    time.sleep(1.5)
    
    property_detail_list.append(pd_json)
    

In [26]:
# Convert to a dataframe

df_property_details = pd.json_normalize(property_detail_list)
df_property_details.head(2)


,listingProvider,buildingPermits,propertyTaxRate,contact_recipients,longitude,countyFIPS,cityId,timeOnZillow,url,zestimate,imgSrc,zpid,zipcode,livingAreaValue,zestimateLowPercent,isListedByOwner,propertyTypeDimension,streetAddress,county,taxHistory,stateId,countyId,timeZone,homeType,livingAreaUnits,comingSoonOnMarketDate,livingArea,bathrooms,annualHomeownersInsurance,state,rentZestimate,building,brokerId,yearBuilt,brokerageName,dateSold,price,pageViewCount,description,homeStatus,homeFacts,latitude,datePosted,bedrooms,nearbyHomes,monthlyHoaFee,priceHistory,favoriteCount,schools,zestimateHighPercent,mlsid,city,providerListingID,country,currency,contingentListingType,solarPotential.sunScore,solarPotential.solarFactor,solarPotential.buildFactor,solarPotential.climateFactor,solarPotential.electricityFactor,resoFacts.hasAttachedProperty,resoFacts.frontageType,resoFacts.poolFeatures,resoFacts.flooring,resoFacts.foundationDetails,resoFacts.accessibilityFeatures,resoFacts.hasGarage,resoFacts.hasPetsAllowed,resoFacts.bodyType,resoFacts.topography,resoFacts.landLeaseExpirationDate,resoFacts.hasAdditionalParcels,resoFacts.waterViewYN,resoFacts.atAGlanceFacts,resoFacts.offerReviewDate,resoFacts.aboveGradeFinishedArea,resoFacts.buyerAgencyCompensationType,resoFacts.rooms,resoFacts.carportSpaces,resoFacts.cityRegion,resoFacts.mainLevelBathrooms,resoFacts.hasPrivatePool,resoFacts.associationFeeIncludes,resoFacts.waterSource,resoFacts.coveredSpaces,resoFacts.inclusions,resoFacts.hasFireplace,resoFacts.zoningDescription,resoFacts.garageSpaces,resoFacts.sewer,resoFacts.elevationUnits,resoFacts.exclusions,resoFacts.mainLevelBedrooms,resoFacts.numberOfUnitsVacant,resoFacts.hasWaterfrontView,resoFacts.listAOR,resoFacts.lotSize,resoFacts.entryLevel,resoFacts.irrigationWaterRightsAcres,resoFacts.greenWaterConservation,resoFacts.stories,resoFacts.livingArea,resoFacts.numberOfUnitsInCommunity,resoFacts.listingTerms,resoFacts.otherStructures,resoFacts.associationFee,resoFacts.marketingType,resoFacts.greenIndoorAirQuality,resoFacts.greenSustainability,resoFacts.hasHomeWarranty,resoFacts.associationPhone,resoFacts.greenBuildingVerificationType,resoFacts.elementarySchool,resoFacts.bedrooms,resoFacts.architecturalStyle,resoFacts.listingId,resoFacts.structureType,resoFacts.interiorFeatures,resoFacts.horseAmenities,resoFacts.electric,resoFacts.developmentStatus,resoFacts.lotFeatures,resoFacts.roofType,resoFacts.waterSources,resoFacts.constructionMaterials,resoFacts.fireplaceFeatures,resoFacts.doorFeatures,resoFacts.appliances,resoFacts.daysOnZillow,resoFacts.bathroomsPartial,resoFacts.bathroomsHalf,resoFacts.depositsAndFees,resoFacts.canRaiseHorses,resoFacts.buildingName,resoFacts.hasAssociation,resoFacts.petsMaxWeight,resoFacts.commonWalls,resoFacts.totalActualRent,resoFacts.additionalParcelsDescription,resoFacts.associationName,resoFacts.allowedPets,resoFacts.communityFeatures,resoFacts.tenantPays,resoFacts.middleOrJuniorSchool,resoFacts.lotSizeDimensions,resoFacts.roadSurfaceType,resoFacts.hasOpenParking,resoFacts.yearBuilt,resoFacts.livingAreaRangeUnits,resoFacts.irrigationWaterRightsYN,resoFacts.bathroomsFull,resoFacts.horseYN,resoFacts.incomeIncludes,resoFacts.highSchool,resoFacts.leaseTerm,resoFacts.elevation,resoFacts.cropsIncludedYN,resoFacts.attic,resoFacts.subdivisionName,resoFacts.windowFeatures,resoFacts.ownership,resoFacts.woodedArea,resoFacts.middleOrJuniorSchoolDistrict,resoFacts.associationPhone2,resoFacts.buyerAgencyCompensation,resoFacts.heating,resoFacts.frontageLength,resoFacts.virtualTourURLUnbranded,resoFacts.associationAmenities,resoFacts.propertySubType,resoFacts.hasCooling,resoFacts.belowGradeFinishedArea,resoFacts.hasElectricOnProperty,resoFacts.foundationArea,resoFacts.zoning,resoFacts.hoaFee,resoFacts.livingAreaRange,resoFacts.basementYN,resoFacts.availabilityDate,resoFacts.waterBodyName,resoFacts.waterfrontFeatures,resoFacts.specialListingConditions,resoFacts.buildingAreaSource,resoFacts.taxAnnualAmount,resoFacts.parking,resoFact

In [27]:
# Highlight the colomns of interest that will be retained and saved on a CSV

detail_cols = ['streetAddress', 
 'city',
 'county',
 'zipcode',
 'state',
 'price',
 'homeType',
 'timeOnZillow', 
 'zestimate',
 'rentZestimate',
 'livingArea',
 'bedrooms',
 'bathrooms',
 'yearBuilt',
 'description',
 'priceHistory',
 'taxHistory',
 'zpid'
 ]

prop_output = df_property_details[detail_cols]
prop_output.head(2)


,streetAddress,city,county,zipcode,state,price,homeType,timeOnZillow,zestimate,rentZestimate,livingArea,bedrooms,bathrooms,yearBuilt,description,priceHistory,taxHistory,zpid
0,4621 Hoste Way Ln,Charlotte,None,28216,NC,360000,SINGLE_FAMILY,5 days,425000.0,2128.0,2521,4,3,2001,Are you looking for a spacious home in a hot n...,"[{'priceChangeRate': 0.38462071007965, 'date':...","[{'time': 1621833645744, 'valueIncreaseRate': ...",64999714
1,425 Heflin St,Charlotte,None,28205,NC,300000,SINGLE_FAMILY,3 days,381000.0,1349.0,1454,3,2,2019,Looking for quality const?You have found it in...,"[{'priceChangeRate': 3.1758241758242, 'date': ...","[{'time': 1621833853375, 'valueIncreaseRate': ...",299908304


In [34]:
# Write to CSV file
with open('propanalysis.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(data)
prop_output.to_csv('propanalysis.csv')

